# Content
- wrangel dataset
- Create a choropleth map
- discussing the results

In [1]:
# import lybaries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
%matplotlib inline

In [3]:
# import data
# I use the dataset with all columns

path = r'/Users/lennartzeidler/Desktop/Uni/(6) Advances Analytics and Dashboard Design/liquid store sales analysis'
sales = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_liquid_sales_clean_all_columns.pkl'))

In [4]:
sales.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
6,S28865700001,11/09/2015,2538,Hy-Vee Food Store #3 / Waterloo,1422 FLAMMANG DR,WATERLOO,50702,"1422 FLAMMANG DR\nWATERLOO 50702\n(42.459938, ...",7.0,Black Hawk,...,238,Forbidden Secret Coffee Pack,6,1500,$11.62,$17.43,6,$104.58,9.0,2.38
8,S29339300091,11/30/2015,2662,Hy-Vee Wine & Spirits / Muscatine,"522 MULBERRY, SUITE A",MUSCATINE,52761,"522 MULBERRY, SUITE A\nMUSCATINE 52761\n",70.0,Muscatine,...,173,Laphroaig w/ Whiskey Stones,12,750,$19.58,$29.37,4,$117.48,3.0,0.79
13,S28866900001,11/11/2015,3650,"Spirits, Stogies and Stuff",118 South Main St.,HOLSTEIN,51025,118 South Main St.\nHOLSTEIN 51025\n(42.490073...,47.0,Ida,...,238,Forbidden Secret Coffee Pack,6,1500,$11.62,$17.43,1,$17.43,1.5,0.40
18,S29134300126,11/18/2015,3723,J D Spirits Liquor,1023 9TH ST,ONAWA,51040,"1023 9TH ST\nONAWA 51040\n(42.025841, -96.095845)",67.0,Monona,...,258,"Rumchata ""GoChatas""",1,6000,$99.00,$148.50,1,$148.50,6.0,1.59
21,S29282800048,11/23/2015,2642,Hy-Vee Wine and Spirits / Pella,512 E OSKALOOSA,PELLA,50219,"512 E OSKALOOSA\nPELLA 50219\n(41.397023, -92....",63.0,Marion,...,238,Forbidden Secret Coffee Pack,6,1500,$11.62,$17.43,6,$104.58,9.0,2.38


### wrangel dataset
I got my Json from https://geodata.iowa.gov/datasets/iowa::iowa-county-boundaries/explore?location=41.962084%2C-93.498741%2C7.59 


In [5]:
# put country number into sales df
sales['Country Number'] = sales['County Number'].astype(int)

In [6]:
sales['County Number'].value_counts()

77.0    2275842
57.0    1049769
82.0     780681
7.0      685137
52.0     626067
         ...   
2.0       10433
93.0      10355
26.0       9677
80.0       9062
36.0       1243
Name: County Number, Length: 99, dtype: int64

In [7]:
# create subset for map
sub = sales[['Sale (Dollars)', 'County Number']]

In [8]:
# cleaning 'Sale (Dollars)' column
sub['Sale (Dollars)'] = sub['Sale (Dollars)'].str.replace("$","")
sub['Sale (Dollars)'] = sub['Sale (Dollars)'].astype(float)

/var/folders/zw/37lckzg54ll7byfmgjl5pkw40000gn/T/ipykernel_16024/1281162340.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sub['Sale (Dollars)'] = sub['Sale (Dollars)'].str.replace("$","")
/var/folders/zw/37lckzg54ll7byfmgjl5pkw40000gn/T/ipykernel_16024/1281162340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['Sale (Dollars)'] = sub['Sale (Dollars)'].str.replace("$","")
/var/folders/zw/37lckzg54ll7byfmgjl5pkw40000gn/T/ipykernel_16024/1281162340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [9]:
sub.head()

,Sale (Dollars),County Number
6,104.58,7.0
8,117.48,70.0
13,17.43,47.0
18,148.50,67.0
21,104.58,63.0


In [10]:
sub['Sale (Dollars)'].describe()

count    1.249597e+07
mean     1.165708e+02
std      3.850013e+02
min      0.000000e+00
25%      2.540000e+01
50%      6.714000e+01
75%      1.327200e+02
max      2.541000e+05
Name: Sale (Dollars), dtype: float64

In [11]:
# excluding outliers
sub = sub[sub['Sale (Dollars)'] < 500]

In [12]:
sub['Sale (Dollars)'].describe()

count    1.225033e+07
mean     8.965088e+01
std      8.364417e+01
min      0.000000e+00
25%      2.475000e+01
50%      6.552000e+01
75%      1.260000e+02
max      4.998600e+02
Name: Sale (Dollars), dtype: float64

In [13]:
# Import ".json" file for the whole world 

country_geo = r'/Users/lennartzeidler/Desktop/Uni/(6) Advances Analytics and Dashboard Design/liquid store sales analysis/02 Data/map-json/Iowa_County_Boundaries.geojson'

In [14]:
# taking a look at the file
f = open('/Users/lennartzeidler/Desktop/Uni/(6) Advances Analytics and Dashboard Design/liquid store sales analysis/02 Data/map-json/Iowa_County_Boundaries.geojson')
data = json.load(f)
leng = 0
for i in data['features']:
    print(i)

{'type': 'Feature', 'properties': {'OBJECTID': 1, 'PERIMETER': 193975.5, 'DOMCountyID': 60, 'FIPS': '19119', 'FIPS_INT': 19119, 'CountyName': 'Lyon', 'StateAbbr': 'IA', 'SHAPE_Length': 2.2127526708553003, 'SHAPE_Area': 0.1691018345380776}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-96.59915441799996, 43.500333546000036], [-96.58646743099996, 43.500300485000025], [-96.56643811899994, 43.50023442100007], [-96.54643406899999, 43.50020006400007], [-96.52654343799998, 43.500158690000035], [-96.52404983099996, 43.50015462500005], [-96.50653798199994, 43.500333451000074], [-96.50060923499996, 43.500389262000056], [-96.48654343699997, 43.50040510800005], [-96.46656928699997, 43.500292469000044], [-96.44674280099997, 43.50029792200007], [-96.44659578999995, 43.50030235600008], [-96.42979997999998, 43.50029445100006], [-96.40981507599997, 43.500298232000034], [-96.40967674099994, 43.500302359000045], [-96.389988814, 43.50028479800005], [-96.37560426999994, 43.50028739100003], [-96.

limit_output extension: Maximum message size of 10000 exceeded with 1446720 characters

### Create a choropleth map

In [17]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = sub,
    columns = ['County Number', 'Sale (Dollars)'],
    key_on = 'feature.properties.DOMCountyID', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "rating").add_to(map)
folium.LayerControl().add_to(map)

map

limit_output extension: Maximum message size of 10000 exceeded with 1497234 characters